In [ ]:
pip install azure.cognitiveservices.vision.customvision

In [ ]:
from azure.cognitiveservices.vision.customvision.training import CustomVisionTrainingClient
from azure.cognitiveservices.vision.customvision.prediction import CustomVisionPredictionClient
import matplotlib.pyplot as plt
from msrest.authentication import ApiKeyCredentials
import cv2
import os
import pandas as pd

%matplotlib inline



In [ ]:
publish_iteration_name = "Iteration8"
project_id = "f8674453-7ef5-4e9f-8a7e-a46ac5c9f031"
training_key = "d7eb84c4933942489534689f02cec4c2"
prediction_key = "27d695fc9bb04245a4a9e3c3674f8153"
endpoint = "https://northeurope.api.cognitive.microsoft.com/"


In [ ]:

def append_predictions(result, confidence_level = 0.90):
    predictions = []
    for prediction in result.predictions:
        if prediction.probability > confidence_level:
            predictions.append(prediction)
                       
    return predictions


def add_boxes_to_images(img, predictions):
    color = 'white' 
    for pred in predictions:
        x = int(pred.bounding_box.left * img.shape[0])
        y = int(pred.bounding_box.top * img.shape[1])
        width = x + int(pred.bounding_box.width * img.shape[0])
        height = y + int(pred.bounding_box.height * img.shape[1])

        img = cv2.rectangle(img, (x, y), (width, height), (0, 0, 255), 2)
        img = cv2.putText(img, str(pred.probability*100) , (x + 5, y + 20), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,0,255), 1, cv2.LINE_AA, False) 
        
            
     

def show_inline_img(img):
    inline_img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)    
    plt.figure(figsize=(10, 20))
    plt.axis('off')    
    plt.imshow(inline_img)

In [ ]:
training_credentials = ApiKeyCredentials(in_headers={"Training-key": training_key})
train  = CustomVisionPredictionClient(endpoint, training_credentials)


prediction_credentials = ApiKeyCredentials(in_headers={"Prediction-key": prediction_key})
predict = CustomVisionPredictionClient(endpoint, prediction_credentials)


In [ ]:
df = pd.DataFrame(columns=['Coords', 'Solar Panel Prediction Percentage'])

folder_dir = './Img/' 
confidence_level = 0.90
 
for image in os.listdir(folder_dir):
 
    if (image.endswith(".png")):  
        image_path = os.path.join(folder_dir, image);      
       
        with open(image_path ,mode='rb') as test_img:
            result = predict.detect_image(project_id, publish_iteration_name, test_img) 
            
            for prediction in result.predictions:
                if prediction.probability > confidence_level:
                    predictions = append_predictions(result)  
                    img = cv2.imread(image_path, cv2.IMREAD_COLOR)
                    img = cv2.resize(img, (400, 400))
                    img.shape
                    add_boxes_to_images(img, predictions)
                    values_to_add = {'Coords': image, 'Solar Panel Prediction Percentage': prediction.probability*100}
                    row_to_add = pd.Series(values_to_add, name=0)
                    df = df.append(row_to_add)                  
                    cv2.imwrite('./Results/'+ image, img)
                    

                    
print(df)         

In [ ]:
df = pd.DataFrame(columns=['Coords', 'Solar Panel Prediction Percentage'])

folder_dir = './Img/' 
confidence_level = 0.90
 
for image in os.listdir(folder_dir):
 
    if (image.endswith(".png")):  
        image_path = os.path.join(folder_dir, image);      
       
        with open(image_path ,mode='rb') as test_img:
            result = predict.detect_image(project_id, publish_iteration_name, test_img) 
            
            for prediction in result.predictions:
                if prediction.probability > confidence_level:
                    values_to_add = {'Coords': image, 'Solar Panel Prediction Percentage': prediction.probability*100}
                    row_to_add = pd.Series(values_to_add, name=0)
                    df = df.append(row_to_add)                  
            
            img = cv2.imread(image_path, cv2.IMREAD_COLOR)
            img = cv2.resize(img, (400, 400))
            predictions = append_predictions(result)  
            add_boxes_to_images(img, predictions)
            img.shape
            cv2.imwrite('./Results/'+ image, img)


                    

                    
print(df)         